# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

NameError: name 'MongoClient' is not defined

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

NameError: name 'mongo' is not defined

In [ ]:
# confirm that our new database was created
db_names = mongo.list_database_names()
if 'uk_food' in db_names:
    print("Database 'uk_food' exists.")
else:
    print("Database 'uk_food' does not exist yet.")


In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections:", collections)

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections:", collections)

In [ ]:
# review a document in the establishments collection
sample_document = establishments.find_one()
pprint(sample_document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Example Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "EX1 2MP",
    "Phone": "0123456789",
    "RatingValue": None,  # Not rated yet
    "RatingDate": None,
    "LocalAuthorityName": "Greenwich"
}

In [ ]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
print("Inserted ID:", insert_result.inserted_id)

In [ ]:
# Check that the new restaurant was inserted
inserted_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(inserted_restaurant)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_id = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
pprint(business_type_id)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
if business_type_id:
    update_result = establishments.update_one(
        {"BusinessName": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id["BusinessTypeID"]}}
    )
    print("Updated Count:", update_result.modified_count)

In [ ]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Dover Count:", dover_count)


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print("Deleted Count:", delete_result.deleted_count)

In [ ]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Remaining Dover Count:", remaining_dover_count)

In [ ]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
pprint(remaining_document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [{"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}, "geocode.latitude": {"$toDecimal": "$geocode.latitude"}}}]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

In [ ]:
# Check that the coordinates and rating value are now numbers
coordinates_check = establishments.find_one({"BusinessName": "Penang Flavours"}, {"geocode.longitude": 1, "geocode.latitude": 1})
rating_value_check = establishments.find_one({"BusinessName": "Penang Flavours"}, {"RatingValue": 1})

pprint(coordinates_check)
pprint(rating_value_check)